# Sentiment analysis
1. Task type: NLP
2. Dataset: Tweets (Text type)
3. Usecases: Social Media management, Review Systems, News Analysis for Stock Markets

In [114]:
import pandas as pd
import numpy as np

In [115]:
df = pd.read_csv('./data/train.csv', encoding='latin-1', names=['Target', 'TweetID', 'Date', 'No_Query', 'UserName', 'Data'])
df.head()

Observation: 
1. Data is not utf-8 encoded that is why required to set correct encoding to read csv file.
2. Data is not having column names that is why provided it with column name.

In [116]:
df.info()

As it is visible to me that Target and Data are only columns useful for me to train model for sentiment detection, I can drop other columns.

In [117]:
df = df[['Target', 'Data']]
df.head()

Now, I will try to remove words which usually does not contribute to sentiments like tags (@username in data in tweet) and urls. I will keep hashtags as of now to check if they make any effect on data or not. 

In [118]:
df['Data'] = df['Data'].replace(r'http\S+', '', regex=True).replace(r'@\S+', '', regex=True)

In [119]:
df.head(20)

Data info showing data is not having null values and datatypes are int64 or objects. Now I need to determine language of text for each statement as I want my model to get trained for english only. 

In [120]:
from langdetect import detect

In [121]:
from numpy import NaN


for i in range(len(df)):
    if df['Data'][i].isspace() == True:
        df['Data'][i] = NaN

In [122]:
df = df[df['Data'].noatna()]

In [123]:
df = df.reset_index(drop=True)

In [125]:
import string
for char in string.punctuation:
    df['Data'] = df['Data'].replace(char, NaN, regex=False)

In [126]:
df.head()

In [127]:
df['ln']=[0]*len(df)
print(df.head())
for i in range(len(df)):
    try:
        x = detect(df['Data'][i])
        df['ln'][i] = x
    except:
        df['ln'][i]=NaN

for i in range(len(df)):
    if df['ln'][i]=='en':
        df['ln'][i]='en'
    else:
        df['ln'][i]=NaN

In [128]:
df.head(20)

In [129]:
df.to_csv('./data/trainModified.csv')

In [130]:
df = pd.read_csv('./data/trainModified.csv')
df.head(10)

,Unnamed: 0,Unnamed: 0.1,Target,Data,ln
0,0,0,0,"- Awww, that's a bummer. You shoulda got Da...",en
1,1,1,0,is upset that he can't update his Facebook by ...,en
2,2,2,0,I dived many times for the ball. Managed to s...,en
3,3,3,0,my whole body feels itchy and like its on fire,en
4,4,4,0,"no, it's not behaving at all. i'm mad. why am...",en
5,5,5,0,not the whole crew,en
6,6,6,0,Need a hug,en
7,7,7,0,"hey long time no see! Yes.. Rains a bit ,onl...",en
8,8,8,0,nope they didn't have it,en
9,9,9,0,que me muera ?,NaN


In [131]:
df = df[df.ln == 'en']
# df = df.drop(df.iloc[:, 0:1], axis=1)
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df = df.reset_index()
df.head(10)

,index,Target,Data,ln
0,0,0,"- Awww, that's a bummer. You shoulda got Da...",en
1,1,0,is upset that he can't update his Facebook by ...,en
2,2,0,I dived many times for the ball. Managed to s...,en
3,3,0,my whole body feels itchy and like its on fire,en
4,4,0,"no, it's not behaving at all. i'm mad. why am...",en
5,5,0,not the whole crew,en
6,6,0,Need a hug,en
7,7,0,"hey long time no see! Yes.. Rains a bit ,onl...",en
8,8,0,nope they didn't have it,en
9,10,0,spring break in plain city... it's snowing,en


In [132]:
len(df[df.Target==0])/len(df[df.Target==4])

1.0135628243231949

It is now clearly visible that data is pretty cleaned and sample is almost of similar length so we can train model without thinking much about bias due to unbalanced data. Data length ratio of 1.01 suggests almost same length of both targets.

In [134]:
df = df[['Data','Target']]
df.head()

,Data,Target
0,"- Awww, that's a bummer. You shoulda got Da...",0
1,is upset that he can't update his Facebook by ...,0
2,I dived many times for the ball. Managed to s...,0
3,my whole body feels itchy and like its on fire,0
4,"no, it's not behaving at all. i'm mad. why am...",0


In [135]:
df['Target'].value_counts()

0    739687
4    729789
Name: Target, dtype: int64

Now, I trained model on complete data and it was taking too much time 

In [137]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Data,Target
0,yeh same here. if you learn how to run magent...,4
1,don't!!! cuz I don't that's bad for you,0
2,The hills! I love it,4
3,'Twas a success! And I ran into an old friend.,4
4,trying to fall asleep without Kyle next to me,0


In [139]:
df = df.drop(df.index[800000:])

In [142]:
tweet = df.Data.values

In [143]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(tweet)

In [144]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [145]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [146]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size = len(tokenizer.word_index) + 1
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           6326912   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 6,343,563
Trainable params: 6,343,563
Non-trainable params: 0
____________________________________________

In [147]:
sentiment_label = df.Target.factorize()

In [153]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.5, epochs=5, batch_size=800)

Epoch 1/5
500/500 [==============================] - 1226s 2s/step - loss: 0.4469 - accuracy: 0.7933 - val_loss: 0.4335 - val_accuracy: 0.7981
Epoch 2/5
500/500 [==============================] - 1291s 3s/step - loss: 0.4387 - accuracy: 0.7970 - val_loss: 0.4279 - val_accuracy: 0.8009
Epoch 3/5
500/500 [==============================] - 1259s 3s/step - loss: 0.4325 - accuracy: 0.7992 - val_loss: 0.4260 - val_accuracy: 0.8028
Epoch 4/5
500/500 [==============================] - 1180s 2s/step - loss: 0.4273 - accuracy: 0.8022 - val_loss: 0.4213 - val_accuracy: 0.8046
Epoch 5/5
500/500 [==============================] - 1154s 2s/step - loss: 0.4235 - accuracy: 0.8040 - val_loss: 0.4203 - val_accuracy: 0.8058


In [158]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    if sentiment_label[1][prediction]==0:
        print('Predicted Sentiment: Negative')
    else:
        print('Predicted Sentiment: Positive')


In [160]:
predict_sentiment('His daughter died in his arms')
predict_sentiment('Life is so good')

1/1 [==============================] - 0s 250ms/step
Predicted Sentiment: Negative
1/1 [==============================] - 0s 229ms/step
Predicted Sentiment: Positive
